In [9]:
import numpy as np
import torch
import torch.optim as optim

In [4]:
x_train = np.load('pipeline_data/x_train_none_none.npz', allow_pickle=True)['data']
x_test = np.load('pipeline_data/x_test_none_none.npz', allow_pickle=True)['data']
y_train = np.load('pipeline_data/y_train.npz', allow_pickle=True)['data']
y_test = np.load('pipeline_data/y_test.npz', allow_pickle=True)['data']

In [5]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((10000, 1000), (3000, 1000), (10000,), (3000,))

In [7]:
N, D_in, D_out = x_train.shape[0], x_train.shape[1], 10
learning_rate = 0.001

x_train = torch.Tensor(x_train)
y_train = np.squeeze(torch.LongTensor(y_train))

In [10]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_out),
)

loss_fn = torch.nn.NLLLoss()
    
losses = []

batchs = 256
    
trainset = torch.utils.data.TensorDataset(x_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batchs,
                                          shuffle=True, num_workers=1)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

batchesn = int(N/batchs)
epochs = 50

In [15]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, datap in enumerate(trainloader, 0):
        inputs, labels = datap
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % batchesn == batchesn-1:
            running_loss = 0.0
    scheduler.step()
  
torch.save(model, 'pipeline_data/nn_benchmark.pt')

In [17]:
model = torch.load('pipeline_data/nn_benchmark.pt', map_location=torch.device('cpu'))

In [18]:
x_test = torch.Tensor(x_test)
y_test = np.squeeze(torch.LongTensor(y_test))

model.eval()
outputs = model(x_test)
probals, predicted = torch.max(outputs, 1)

c = (predicted == y_test).squeeze()
correct = c.sum().item()
print('samples correct={} accuracy={}'.format(correct, np.round(correct/x_test.shape[0], 3)))

samples correct=2744 accuracy=0.915


In [20]:
for set_path in ['pipeline_data/x_test_salt_pepper_noise_0.03.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.06.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.09.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.12.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.15.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.18.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.21.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.24.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.27.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.30.npz',
                 'pipeline_data/x_test_salt_pepper_noise_0.33.npz']:
    x = torch.Tensor(np.load(set_path, allow_pickle=True)['data'])
    y = np.squeeze(torch.LongTensor(
        np.load('pipeline_data/y_test.npz', allow_pickle=True)['data']))

    model.eval()
    outputs = model(x)
    probals, predicted = torch.max(outputs, 1)

    c = (predicted == y).squeeze()
    correct = c.sum().item()
    print('{} {}'.format(set_path, np.round(correct/x.shape[0], 3)))

pipeline_data/x_test_salt_pepper_noise_0.03.npz 0.858
pipeline_data/x_test_salt_pepper_noise_0.06.npz 0.811
pipeline_data/x_test_salt_pepper_noise_0.09.npz 0.771
pipeline_data/x_test_salt_pepper_noise_0.12.npz 0.727
pipeline_data/x_test_salt_pepper_noise_0.15.npz 0.695
pipeline_data/x_test_salt_pepper_noise_0.18.npz 0.647
pipeline_data/x_test_salt_pepper_noise_0.21.npz 0.616
pipeline_data/x_test_salt_pepper_noise_0.24.npz 0.583
pipeline_data/x_test_salt_pepper_noise_0.27.npz 0.536
pipeline_data/x_test_salt_pepper_noise_0.30.npz 0.506
pipeline_data/x_test_salt_pepper_noise_0.33.npz 0.459


In [21]:
for set_path in ['pipeline_data/x_test_gaussian_blur_0.5.npz',
                 'pipeline_data/x_test_gaussian_blur_1.0.npz',
                 'pipeline_data/x_test_gaussian_blur_1.5.npz',
                 'pipeline_data/x_test_gaussian_blur_2.0.npz',
                 'pipeline_data/x_test_gaussian_blur_2.5.npz',
                 'pipeline_data/x_test_gaussian_blur_3.0.npz',
                 'pipeline_data/x_test_gaussian_blur_3.5.npz',
                 'pipeline_data/x_test_gaussian_blur_4.0.npz',
                 'pipeline_data/x_test_gaussian_blur_4.5.npz',
                 'pipeline_data/x_test_gaussian_blur_5.0.npz',
                 'pipeline_data/x_test_gaussian_blur_5.5.npz']:
    x = torch.Tensor(np.load(set_path, allow_pickle=True)['data'])
    y = np.squeeze(torch.LongTensor(
        np.load('pipeline_data/y_test.npz', allow_pickle=True)['data']))

    model.eval()
    outputs = model(x)
    probals, predicted = torch.max(outputs, 1)

    c = (predicted == y).squeeze()
    correct = c.sum().item()
    print('samples correct={} accuracy={}'.format(correct, np.round(correct/x.shape[0], 3)))

samples correct=2743 accuracy=0.914
samples correct=2722 accuracy=0.907
samples correct=2661 accuracy=0.887
samples correct=2602 accuracy=0.867
samples correct=2527 accuracy=0.842
samples correct=2424 accuracy=0.808
samples correct=2331 accuracy=0.777
samples correct=2219 accuracy=0.74
samples correct=2109 accuracy=0.703
samples correct=1990 accuracy=0.663
samples correct=1891 accuracy=0.63


In [22]:
for set_path in ['pipeline_data/x_test_gaussian_noise_10.npz',
                 'pipeline_data/x_test_gaussian_noise_20.npz',
                 'pipeline_data/x_test_gaussian_noise_30.npz',
                 'pipeline_data/x_test_gaussian_noise_40.npz',
                 'pipeline_data/x_test_gaussian_noise_50.npz',
                 'pipeline_data/x_test_gaussian_noise_60.npz',
                 'pipeline_data/x_test_gaussian_noise_70.npz',
                 'pipeline_data/x_test_gaussian_noise_80.npz',
                 'pipeline_data/x_test_gaussian_noise_90.npz',
                 'pipeline_data/x_test_gaussian_noise_100.npz',
                 'pipeline_data/x_test_gaussian_noise_110.npz']:
    x = torch.Tensor(np.load(set_path, allow_pickle=True)['data'])
    y = np.squeeze(torch.LongTensor(
        np.load('pipeline_data/y_test.npz', allow_pickle=True)['data']))

    model.eval()
    outputs = model(x)
    probals, predicted = torch.max(outputs, 1)

    c = (predicted == y).squeeze()
    correct = c.sum().item()
    print('samples correct={} accuracy={}'.format(correct, np.round(correct/x.shape[0], 3)))

samples correct=2746 accuracy=0.915
samples correct=2704 accuracy=0.901
samples correct=2641 accuracy=0.88
samples correct=2550 accuracy=0.85
samples correct=2430 accuracy=0.81
samples correct=2303 accuracy=0.768
samples correct=2156 accuracy=0.719
samples correct=1988 accuracy=0.663
samples correct=1847 accuracy=0.616
samples correct=1698 accuracy=0.566
samples correct=1552 accuracy=0.517
